In [9]:
!pip install datasets

In [10]:
from datasets import load_dataset

dataset = load_dataset("imdb")
train_data = dataset["train"]
test_data = dataset["test"]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [11]:
import re
from collections import Counter

def basic_tokenizer(text):
    text = text.lower()
    text = re.sub(r"[^a-z0-9]+", " ", text)
    return text.split()

counter = Counter()
for item in train_data:
    counter.update(basic_tokenizer(item["text"]))

vocab = {"<pad>": 0, "<unk>": 1}
for word, _ in counter.most_common(20000):
    vocab[word] = len(vocab)

def encode(text):
    return [vocab.get(tok, vocab["<unk>"]) for tok in basic_tokenizer(text)]


In [12]:
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_fn(batch):
    xs, ys = [], []
    for item in batch:
        xs.append(torch.tensor(encode(item["text"]), dtype=torch.long))
        ys.append(1 if item["label"] == 1 else 0)
    xs = pad_sequence(xs, batch_first=True, padding_value=vocab["<pad>"])
    ys = torch.tensor(ys, dtype=torch.long)
    return xs.to(device), ys.to(device)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True, collate_fn=collate_fn)
test_loader  = DataLoader(test_data,  batch_size=32, shuffle=False, collate_fn=collate_fn)


In [13]:
import torch.nn as nn

class GRUClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=128):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=vocab["<pad>"])
        self.gru = nn.GRU(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 2)

    def forward(self, x):
        emb = self.embedding(x)
        out, h = self.gru(emb)
        return self.fc(h[-1])

model = GRUClassifier(len(vocab)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


In [14]:
from tqdm import tqdm

def train_epoch():
    model.train()
    total_loss = 0
    for x, y in tqdm(train_loader):
        optimizer.zero_grad()
        pred = model(x)
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

def evaluate():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y in test_loader:
            pred = model(x).argmax(1)
            correct += (pred == y).sum().item()
            total += y.size(0)
    return correct / total


In [ ]:
for epoch in range(3):
    loss = train_epoch()
    acc = evaluate()
    print(f"Epoch {epoch+1}: Loss={loss:.4f}, Test Acc={acc:.4f}")


  3%|▎         | 27/782 [00:56<25:53,  2.06s/it]

In [ ]:
def predict(text):
    model.eval()
    x = torch.tensor(encode(text)).unsqueeze(0).to(device)
    pred = model(x).argmax(1).item()
    return "Positive" if pred == 1 else "Negative"

print(predict("This movie is really great, I loved it."))
print(predict("Terrible movie, boring and slow."))
